Assignement: Using the data provided below build a Fake News detecting models with Python.

About the Dataset:


    1. title: the title of a news article         
    2. text: the title of a news article         
    3. label: a label that marks whether the news article is Real or Fake

In [45]:
#Importing the Dependecies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [46]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/k3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
#Print Stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [48]:
# Data Preprocessing

news_df = pd.read_csv('news.csv')

In [49]:
#Shape of our datset

news_df.shape

(6335, 4)

In [50]:
#First 10 rows of the dataframe
news_df.head(10)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [51]:
#Checking and counting the number of missing values in the dataset
news_df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [52]:
news_df['content'] = news_df['title']+ news_df['text']
print(news_df['content'])

0       You Can Smell Hillary’s FearDaniel Greenfield,...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathyU.S...
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 6335, dtype: object


In [53]:
#Spliting the Data and the label
X = news_df.drop(columns='label', axis=1)
Y = news_df['label']


print(Y)
print(X)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object
      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Sudde

Stemmin:

Stemming is the process of reducing a word to its  Root word.

example: 
actor, actress, acting --> act

In [54]:
port_stem = PorterStemmer()

In [55]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [56]:
news_df['content'] = news_df['content'].apply(stemming)

In [ ]:
print(news_df['content'])

0       smell hillari feardaniel greenfield shillman j...
1       watch exact moment paul ryan commit polit suic...
2       kerri go pari gestur sympathyu secretari state...
3       berni support twitter erupt anger dnc tri warn...
4       battl new york primari mattersit primari day n...
                              ...                        
6330    state depart say find email clinton specialist...
6331    p pb stand plutocrat pentagon p pb stand pluto...
6332    anti trump protest tool oligarchi inform anti ...
6333    ethiopia obama seek progress peac secur east a...
6334    jeb bush suddenli attack trump mattersjeb bush...
Name: content, Length: 6335, dtype: object


In [ ]:
#separating the data and label
X = news_df['content'].values
Y = news_df['label'].values

In [ ]:
print(X)

['smell hillari feardaniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scan

In [ ]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [ ]:
Y.shape

(6335,)

In [ ]:
X.shape

(6335,)

In [ ]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 47511)	0.03038602655614134
  (0, 47354)	0.018250407048591665
  (0, 47038)	0.03697738116152465
  (0, 47034)	0.024011455425876467
  (0, 47007)	0.033104887479806555
  (0, 46974)	0.02362379518816086
  (0, 46972)	0.008740021548713141
  (0, 46880)	0.015809424537374484
  (0, 46816)	0.016682687117467867
  (0, 46728)	0.0283947456922022
  (0, 46725)	0.016820013829032483
  (0, 46683)	0.03827775522541423
  (0, 46619)	0.04262121306726082
  (0, 46509)	0.0348943117534179
  (0, 46444)	0.035819312515813335
  (0, 46429)	0.018404668072519365
  (0, 46349)	0.01392472590517313
  (0, 46316)	0.061966926286064046
  (0, 46208)	0.011506025190865545
  (0, 46175)	0.027023124574181708
  (0, 46091)	0.028676965839198633
  (0, 46022)	0.03273948226701235
  (0, 45968)	0.01637650498268942
  (0, 45901)	0.053343324513146904
  (0, 45860)	0.07043372811319827
  :	:
  (6334, 4237)	0.02870644155063649
  (6334, 4236)	0.026528196477111597
  (6334, 4038)	0.035466457163606004
  (6334, 4030)	0.020567481687402574
  (6334, 3731)

Splitting the datasey to training and text data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify =Y, random_state=2)

Training the Model: Logistic Refression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)


In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9530386740331491


In [ ]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9171270718232044


Making a Predictive System

In [ ]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] ==0):
    print('The news is Real')
else:
    print('The news is Fake')

['FAKE']
The news is Fake


In [ ]:
print(Y_test[1])

FAKE
